In [10]:
from google.cloud import videointelligence
import argparse
import yaml
import pandas as pd
from IPython.display import display, HTML

# Read parameter from yaml file.
with open("params.yaml") as file:
    yaml_file = yaml.load(file, Loader=yaml.FullLoader)

gcs_video_path = yaml_file["video"]["sample"]

def analyze_shots(path):
    """ Detects camera shot changes. 
    
    Parameters:
        path: Google Cloud Storage path.
        
    Return:
        dictionary: Ex {"index": [1,2,3], "start_time"..., "end_time": ...}
    """

    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.enums.Feature.SHOT_CHANGE_DETECTION]
    operation = video_client.annotate_video(gcs_video_path, features=features)
    print('\nProcessing video for shot change annotations:')

    result = operation.result(timeout=120)

    indexes = []
    start_times = []
    end_times = []
    for i, shot in enumerate(result.annotation_results[0].shot_annotations):

        start_time = (shot.start_time_offset.seconds + shot.start_time_offset.nanos / 1e9)
        end_time = (shot.end_time_offset.seconds + shot.end_time_offset.nanos / 1e9)

        print('\tShot {}: {} to {}'.format(i, start_time, end_time))
        
        indexes.append(i)
        start_times.append(start_time)
        end_times.append(end_time)

    return {"index": indexes, "start_time": start_times, "end_time": end_times}

result = analyze_shots(gcs_video_path)


Processing video for shot change annotations:

Finished processing. annotation_results {
  input_uri: "/vision-ai-experimentation/videos/botanic_garden_video.mov"
  shot_annotations {
    start_time_offset {
    }
    end_time_offset {
      seconds: 2
      nanos: 733333000
    }
  }
  shot_annotations {
    start_time_offset {
      seconds: 2
      nanos: 766666000
    }
    end_time_offset {
      seconds: 4
      nanos: 400000000
    }
  }
  shot_annotations {
    start_time_offset {
      seconds: 4
      nanos: 433333000
    }
    end_time_offset {
      seconds: 8
      nanos: 366666000
    }
  }
  shot_annotations {
    start_time_offset {
      seconds: 8
      nanos: 400000000
    }
    end_time_offset {
      seconds: 19
      nanos: 366666000
    }
  }
  shot_annotations {
    start_time_offset {
      seconds: 19
      nanos: 400000000
    }
    end_time_offset {
      seconds: 22
      nanos: 333333000
    }
  }
  shot_annotations {
    start_time_offset {
      seconds

In [9]:
# Display DataFrame as Table.
df = pd.DataFrame({
    "index": result["index"],
    "start_time": result["start_time"],
    "end_times": result["end_times"],
})

display(df)

,index,start_time,end_times
0,0,0.000000,2.733333
1,1,2.766666,4.400000
2,2,4.433333,8.366666
3,3,8.400000,19.366666
4,4,19.400000,22.333333
5,5,22.366666,26.366666
6,6,26.400000,27.366666
7,7,27.400000,28.366666
8,8,28.400000,46.366666
9,9,46.400000,47.366666
